# Client Side Encryption

This feature allows you to use your own encryption key to encrypt your data before it is sent to the DynamoDB.

NOTE: this solution is based on [pynamodb_mate](https://github.com/MacHu-GWU/pynamodb_mate-project) Python library.

**Summary**

DynamoDB support encryption at the rest (Server Side Encryption) and use SSL to encryption the transit data (Encrypt at the fly) by default.

Some advanced user also wants to encrypt the data before it is sent to the DynamoDB (Client Side Encryption). This pattern is not that easy to implement it right.

**How it works**

``pynamodb_mate`` uses the `pycryptodome <https://pypi.org/project/pycryptodome/>`_ crypto library under the hood.
serialize
Internally it always serializes your data into binary, and encrypt it, then send to DynamoDB. For field that you still want to be able to query on it, you use ``determinative = True``. The same input data will always become the same encrypted data, and it uses AES ECB. It is proved that not secure for middle man attack, but you can still use it with DynamoDB because DynamoDB api use SSL to encrypt it in transit. For ``determinative = False``, the same input will become different encrypted data, it uses AES CTR.

##  Define attribute to use Client Side Encryption (AES)

In [1]:
import pynamodb_mate as pm

ENCRYPTION_KEY = "my-password"

class ArchiveModel(pm.Model):
    class Meta:
        table_name = f"pynamodb-mate-example-client-side-encryption"
        region = "us-east-1"
        billing_mode = pm.PAY_PER_REQUEST_BILLING_MODE

    aid = pm.UnicodeAttribute(hash_key=True)

    secret_message = pm.EncryptedUnicodeAttribute(
        # the field level encryption key
        encryption_key=ENCRYPTION_KEY,
        # if True, same input -> same output (less secure),
        # so you can still use this field for query
        # ``filter_conditions=(ArchiveModel.secret_message == "my message")``.
        # if False, same input -> different output (more secure),
        # but you lose the capability of query on this field
        determinative=True,
    )

    secret_binary = pm.EncryptedBinaryAttribute(
        encryption_key=ENCRYPTION_KEY,
        determinative=False,
    )

    secret_integer = pm.EncryptedNumberAttribute(
        encryption_key=ENCRYPTION_KEY,
        determinative=True,
    )

    secret_float = pm.EncryptedNumberAttribute(
        encryption_key=ENCRYPTION_KEY,
        determinative=False,
    )

    secret_data = pm.EncryptedJsonDictAttribute(
        encryption_key=ENCRYPTION_KEY,
        determinative=False,
    )

# create DynamoDB table if not exists, quick skip if already exists
ArchiveModel.create_table(wait=True)

In [2]:
msg = "attack at 2PM tomorrow!"
binary = "a secret image".encode("utf-8")
data = {"Alice": 1, "Bob": 2, "Cathy": 3}
model = ArchiveModel(
    aid="aid-001",
    secret_message=msg,
    secret_binary=binary,
    secret_integer=1234,
    secret_float=3.14,
    secret_data=data,
)
model.save()
print(f"preview the DynamoDB item: {model.item_detail_console_url}")
print("you will see that the raw data in DynamoDB is encrypted")

preview the DynamoDB item: https://us-east-1.console.aws.amazon.com/dynamodbv2/home?region=us-east-1#edit-item?table=pynamodb-mate-example-client-side-encryption&itemMode=2&pk=aid-001&sk&ref=%23item-explorer%3Ftable%3Dpynamodb-mate-example-client-side-encryption&route=ROUTE_ITEM_EXPLORER
you will see that the raw data in DynamoDB is encrypted


In [3]:
model = ArchiveModel.get("aid-001")
print(model.to_dict())

{'aid': 'aid-001', 'secret_binary': b'a secret image', 'secret_data': {'Alice': 1, 'Bob': 2, 'Cathy': 3}, 'secret_float': 3.14, 'secret_integer': 1234, 'secret_message': 'attack at 2PM tomorrow!'}


In [4]:
# for determinative field, you can still use it for query
for item in ArchiveModel.scan(
        ArchiveModel.secret_message == "attack at 2PM tomorrow!"
):
    print(item.to_dict())

{'aid': 'aid-001', 'secret_binary': b'a secret image', 'secret_data': {'Alice': 1, 'Bob': 2, 'Cathy': 3}, 'secret_float': 3.14, 'secret_integer': 1234, 'secret_message': 'attack at 2PM tomorrow!'}


In [5]:
# Update
model.update([
    ArchiveModel.secret_message.set("Hold the fire now!")
])
model.refresh()

print(model.to_dict())

{'aid': 'aid-001', 'secret_binary': b'a secret image', 'secret_data': {'Alice': 1, 'Bob': 2, 'Cathy': 3}, 'secret_float': 3.14, 'secret_integer': 1234, 'secret_message': 'Hold the fire now!'}


## Custom Encrypted Attribute.

``pynamodb_mate`` has four built-in Encrypted attribute:

- ``pynamodb_mate.EncryptedNumberAttribute``
- ``pynamodb_mate.EncryptedUnicodeAttribute``
- ``pynamodb_mate.EncryptedBinaryAttribute``
- ``pynamodb_mate.EncryptedJsonDictAttribute``

Please take a look at the "Custom S3Backed Attribute" section in [Store Large Object in DynamoDB](https://github.com/MacHu-GWU/pynamodb_mate-project/blob/master/examples/store-large-object.ipynb) for more details.

In [6]:
# A Custom Encrypted Attribute that store Json dict
import json

class EncryptedJsonDictAttribute(pm.SymmetricEncryptedAttribute):
    """
    Encrypted JSON data Attribute.
    """
    def user_serializer(self, value: dict) -> bytes:
        return json.dumps(value).encode("utf-8")

    def user_deserializer(self, value: bytes) -> dict:
        return json.loads(value.decode("utf-8"))